In [1]:
# imports
import numpy as np
import pandas as pd 
import os
from PIL import Image #image processing library 
import joblib

# tensorflow imports for layers models and optimiser
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, Flatten,BatchNormalization, Activation, ZeroPadding2D, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam

from gandywarhol.data_gen import CustomDataGen # our custom data loader
from tqdm.notebook import tqdm # makes progress bar
from tensorflow.keras.utils import plot_model # allows picture view of model to see input and output size of layers

In [2]:
IMAGE_SIZE = 128 # length and width of image
IMAGE_CHANNELS = 3 # image depth (RGB)
# Preview image Frame
PREVIEW_ROWS = 4 # rows of images to be produced
PREVIEW_COLS = 7 # columns of images to be produced therefore 28 images
PREVIEW_MARGIN = 4 # 4 pixels between image 

# how often to save images
SAVE_FREQ = 10

# number of channels of noise to generate images from
NOISE_SIZE = 100

# Configuration
EPOCHS = 500
BATCH_SIZE = 32
GENERATE_RES = 3

In [3]:
path_list = ['../raw_data/'+image for image in os.listdir('../raw_data') if image.endswith('jpeg')]
train_set = CustomDataGen(path_list)

In [4]:
train_set[1].shape

(1, 128, 128, 3)

In [5]:
def build_discriminator(image_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2,
    input_shape=image_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    return Model(input_image, validity)

In [6]:
def build_generator(noise_size, channels):
    model = Sequential()
    model.add(Dense(4 * 4 * 256, activation='relu',       input_dim=noise_size))
    model.add(Reshape((4, 4, 256)))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    for i in range(GENERATE_RES):
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation('relu'))
    model.summary()
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))
    input_ = Input(shape=(noise_size,))
    generated_image = model(input_)
    
    return Model(input_, generated_image)

In [7]:
def save_images(cnt, noise, epoch):
# image_array creates a white rectangle (255) of the size we need it in order to view our 28 images
# The margin is the 4 pixels between each image     
    image_array = np.full((
        PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)),
        PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3),
        255, dtype=np.uint8)
# This generates our 28 images, and then puts them between 0 and 1
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    image_count = 0
# The following loop puts each image in its correct square. r takes the pixel on the far left of each square,
# c takes the pixel at the top of each square and then starting there the image is placed, this loops through 
# for each image and its respective square it belongs. 
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            image_array[r:r + IMAGE_SIZE, c:c +
                        IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1
    output_path = 'output'
# finally we save the picture of all 28 images in our desired place.
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    filename = os.path.join(output_path, f"trained-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)
    fin_img_path = 'single_imgs'
    if not os.path.exists(fin_img_path):
        os.makedirs(fin_img_path)
    if epoch == EPOCHS:
        for i in range(28):
            filename = os.path.join(fin_img_path, f"final-{i}.png")
            im = Image.fromarray(generated_images[i]*255)
            im.save(filename)
            i += 1
# We could edit this slightly so that it saved the epoch number and not just the count of how many images are saved.

In [8]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS) # (128,128,3)

# We use adam optimization
optimizer = Adam(learning_rate = 1.5e-4,beta_1 = 0.5)

# Simple discriminator loss since binary classifier and metric is accuracy since we are looking for accurate results.
discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])


# for the generator we combine it with the disciminator before compiling 

generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)

random_input = Input(shape=(NOISE_SIZE,))
generated_image = generator(random_input)

discriminator.trainable = False

validity = discriminator(generated_image)



combined = Model(random_input, validity)

combined.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])

2022-03-04 16:35:13.420466: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 4096)              413696    
                                                                 
 reshape (Reshape)           (None, 4, 4, 256)         0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 8, 8, 256)        0         
 )                                                               
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 8, 256)         590080    
                                                                 
 batch_normalization_4 (Batc  (None, 8, 8, 256)        1024      
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 8, 8, 256)        

In [9]:
# first we set our y, simply 1 for real images and 0 for fake images
y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE, 1))


# instantiate 28 images of noise since we are creating 28 images with each predict 
# this noise stays fixed so that each image in each position is generated from same noise 
fixed_noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE))
cnt = 1
for epoch in range(EPOCHS):
    for index in range(len(train_set)): # note each iteration of for loop is one batch 
        x_real = train_set[index] # set image between (0,1)
        noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE)) # this is the noise to test our metric so isnt fixed
        x_fake = generator.predict(noise) # create fake image
        if epoch % 2 == 0:
            discriminator_metric_real = discriminator.train_on_batch(x_real, y_real) # how good is discrim at real image
            discriminator_metric_generated = discriminator.train_on_batch(
            x_fake, y_fake) # how good is discrim at detecting fake images
            discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)
        generator_metric = combined.train_on_batch(noise, y_real) # how good is the generator at trying to produce real images 
    train_set.on_epoch_end() # shuffle the images so the next epoch will have different batches 
    if epoch % SAVE_FREQ == 0: # save images at desired intervals 
        save_images(cnt, fixed_noise, epoch)
        cnt += 1

resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling data...
resuffling d

KeyboardInterrupt: 